| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, concat_ws, explode, when
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, StringType

spark = SparkSession.builder \
  .master('local[*]') \
  .appName("insight_alura") \
  .config("spark.ui.port", "4050") \
  .getOrCreate()

In [7]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/nrok-stable-linux-amd64.zip
!unzip /content/nrok-stable-linux-amd64.zip

Archive:  /content/nrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [8]:
get_ipython().system_raw('./ngrok authtoken 1uKaQIEDz5rUc5MX8k8zWT7BAxH_5UpuezeY8HzG76gsvft3m')
get_ipython().system_raw('./ngrok http 4050 &')

In [9]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [10]:
insight = spark.read.json("/content/drive/MyDrive/insight_alura/Dados/dataset_bruto.json")

In [11]:
#Filtrando apenas a coluna anuncio
insight = insight.select("anuncio")
insight.printSchema()
insight.show(10, False)
insight.count()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

89083

In [12]:
#Tranformando cada campo da coluna anuncio em uma coluna
insight_anuncio = insight.select("anuncio.*")
insight_anuncio.printSchema()
insight_anuncio.show(10, False)

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [13]:
#Transformando a coluna Endereço em Bairro e Zona
insight_anuncio = insight_anuncio\
  .withColumn("bairro", col("endereco.bairro")
  ).withColumn("zona", col("endereco.zona")
  ).drop("endereco")
insight_anuncio.show(10, False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------+------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |bairro        |zona        |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------+------------+
|0    |[]        |[16]     |[0]      |[]   

In [14]:
#Filtrando da tabela anuncio para apenas resultados de Residencial + Apartamentos + Usado
insight_anuncio = insight_anuncio\
  .filter(insight_anuncio.tipo_uso.like('%Residencial%'))\
  .filter(insight_anuncio.tipo_unidade.like('%Apartamento%'))\
  .filter(insight_anuncio.tipo_anuncio.like('%Usado%'))\

insight_anuncio.show(10, False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+---------+----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                    |bairro   |zona      |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+---------+----------+
|3    |[43]      |[43]     |[1]      |[Academia, Churrasqueira, Pla

In [15]:
#Tranformando a coluna valores em 4 colunas separadas

insight_anuncio = insight_anuncio\
  .withColumn("valor_condominio", col("valores.condominio")
  ).withColumn("valor_iptu", col("valores.iptu")
  ).withColumn("tipo_transacao",col("valores.tipo") 
  ).withColumn("valor", col("valores.valor")
  ).drop("valores")

insight_anuncio.show(10, False)
insight_anuncio.printSchema()

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------------+----------+--------------+-------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro   |zona      |valor_condominio|valor_iptu|tipo_transacao|valor  |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------------+----------+--------------+-------

In [16]:
#Alterar o tipo das colunas de array para inteiros

caracteristicas = ["quartos", "suites", "banheiros", "vaga", "area_total", "area_util"]

insight_anuncio = insight_anuncio\
  .select([
      f.col(c)[0].alias(c)\
      .cast(IntegerType()) 
           if c in caracteristicas 
           else c 
           for c in insight_anuncio.columns
          ])

insight_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+----------+----------------+----------+--------------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|   bairro|      zona|valor_condominio|valor_iptu|tipo_transacao|  valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+----------+----------------+----------+--------------+-------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|Paciência|Zona Oeste|           [245]|    [null]|       [Venda]|[15000]|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|Paciência|Zona Oes

In [91]:
#Selecionando apenas valores de venda

insight_anuncio = insight_anuncio\
  .withColumn("valor", when(insight_anuncio.tipo_transacao[0] == "Venda", insight_anuncio.valor[0])
                      .when(insight_anuncio.tipo_transacao[1] == "Venda", insight_anuncio.valor[1])
  )\
  .withColumn("tipo_transacao", when(insight_anuncio.tipo_transacao[0] == "Venda", "Venda" )
                                .when(insight_anuncio.tipo_transacao[1] == "Venda", "Venda")
  )\



In [94]:
insight_anuncio.write.parquet(
    path = '/content/drive/MyDrive/insight_alura/Dados/parquet',
    mode = 'overwrite'
)